In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tqdm import tqdm

In [2]:
data_path = 'Bike-Sharing-Dataset/hour.csv'

rides = pd.read_csv(data_path)

In [3]:
rides.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [4]:
dummy_fields = ['season', 'weathersit', 'mnth', 'hr', 'weekday']
for each in dummy_fields:
    dummies = pd.get_dummies(rides[each], prefix=each, drop_first=False)
    rides = pd.concat([rides, dummies], axis=1)

fields_to_drop = ['instant', 'dteday', 'season', 'weathersit', 
                  'weekday', 'atemp', 'mnth', 'workingday', 'hr']
data = rides.drop(fields_to_drop, axis=1)
data.head()

,yr,holiday,temp,hum,windspeed,casual,registered,cnt,season_1,season_2,...,hr_21,hr_22,hr_23,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,0,0,0.24,0.81,0.0,3,13,16,1,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0.22,0.80,0.0,8,32,40,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0.22,0.80,0.0,5,27,32,1,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0.24,0.75,0.0,3,10,13,1,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0.24,0.75,0.0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
quant_features = ['casual', 'registered', 'cnt', 'temp', 'hum', 'windspeed']
# Store scalings in a dictionary so we can convert back later
scaled_features = {}
for each in quant_features:
    mean, std = data[each].mean(), data[each].std()
    scaled_features[each] = [mean, std]
    data.loc[:, each] = (data[each] - mean)/std

In [6]:
# Save data for approximately the last 21 days 
test_data = data[-21*24:]

# Now remove the test data from the data set 
data = data[:-21*24]

# Separate the data into features and targets
target_fields = ['cnt', 'casual', 'registered']
features, targets = data.drop(target_fields, axis=1), data[target_fields]
test_features, test_targets = test_data.drop(target_fields, axis=1), test_data[target_fields]

In [7]:
# Hold out the last 60 days or so of the remaining data as a validation set
train_features, train_targets = features[:-60*24], targets[:-60*24]
val_features, val_targets = features[-60*24:], targets[-60*24:]

## Building a single layer neural network with Tensorflow

In [8]:
train_targets.shape

(15435, 3)

In [9]:
val_features.shape

(1440, 56)

In [10]:
val_targets.head()

,cnt,casual,registered
15435,3.580933,-0.054279,4.309096
15436,0.973258,-0.013715,1.170827
15437,0.151813,-0.175970,0.239256
15438,0.559779,0.351359,0.556386
15439,1.105571,0.858407,1.045296


In [11]:
def neural_net(input, num_features):
    # Build input layer
    labels_count = 1

    num_hidden = 8

    weights_i_h = tf.Variable(tf.truncated_normal((num_features, num_hidden)))
    biases_i_h = tf.Variable(tf.zeros(num_hidden))

    weights_h_o = tf.Variable(tf.truncated_normal((num_hidden, labels_count)))
    biases_h_o = tf.Variable(tf.zeros(labels_count))
    
    logits_i_h = tf.matmul(inputs, weights_i_h) + biases_i_h # when to use tf.add?
    sigmoid_i_h = tf.nn.sigmoid(logits_i_h, name=None)

    prediction = tf.matmul(sigmoid_i_h, weights_h_o) + biases_h_o
    return prediction

In [12]:
def train_neural_network(x, num_features):
    prediction = neural_net(x, num_features) #feed-forward
    loss = tf.reduce_mean(tf.squared_difference(prediction, label))
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)    # default learning_rate = 0.001

    # cycles of feedforwards + backprop
    hm_epochs = 10000
    training_fd = {inputs: train_features, label: train_targets["cnt"][:,None]}
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(hm_epochs):      
            _, c = sess.run([optimizer, loss], feed_dict = training_fd)
            if (epoch % 1000 == 0 or epoch == 9999):
                print("Epoch", epoch, "completed out of", hm_epochs, "loss:", c)



In [13]:
num_features = train_features.shape[1]
labels_count = 1
inputs = tf.placeholder(tf.float32, [None, num_features])
label = tf.placeholder(tf.float32, [None, labels_count])
train_neural_network(inputs, num_features)

Epoch 0 completed out of 10000 loss: 3.07567
Epoch 1000 completed out of 10000 loss: 0.381853
Epoch 2000 completed out of 10000 loss: 0.251948
Epoch 3000 completed out of 10000 loss: 0.23161
Epoch 4000 completed out of 10000 loss: 0.220295
Epoch 5000 completed out of 10000 loss: 0.20794
Epoch 6000 completed out of 10000 loss: 0.191656
Epoch 7000 completed out of 10000 loss: 0.174035
Epoch 8000 completed out of 10000 loss: 0.158193
Epoch 9000 completed out of 10000 loss: 0.142858
Epoch 9999 completed out of 10000 loss: 0.124601


## Adding more layers

In [14]:
def two_hidden_layer_nn(input, num_features):
    # Build input layer
    labels_count = 1

    # layer 1
    num_hidden_1 = 8
    weights_i_h1 = tf.Variable(tf.truncated_normal((num_features, num_hidden_1)))
    biases_i_h1 = tf.Variable(tf.zeros(num_hidden_1))
    logits_i_h1 = tf.matmul(inputs, weights_i_h1) + biases_i_h1 # when to use tf.add?
    sigmoid_i_h1 = tf.nn.sigmoid(logits_i_h1, name=None)
    
    #layer 2
    num_hidden_2 = 8
    weights_h1_h2 = tf.Variable(tf.truncated_normal((num_hidden_1, num_hidden_2)))
    biases_h1_h2 = tf.Variable(tf.zeros(num_hidden_2))
    logits_h1_h2 = tf.matmul(sigmoid_i_h1, weights_h1_h2) + biases_h1_h2 # when to use tf.add?
    sigmoid_h1_h2 = tf.nn.sigmoid(logits_h1_h2, name=None)
    
    #output
    weights_h2_o = tf.Variable(tf.truncated_normal((num_hidden_2, labels_count)))
    biases_h2_o = tf.Variable(tf.zeros(labels_count))
    prediction = tf.matmul(sigmoid_h1_h2, weights_h2_o) + biases_h2_o
    
    return prediction

In [15]:
def train_two_hidden_nn(x, num_features):
    prediction = two_hidden_layer_nn(x, num_features) #feed-forward
    loss = tf.reduce_mean(tf.squared_difference(prediction, label))
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)    # default learning_rate = 0.001

    # cycles of feedforwards + backprop
    hm_epochs = 10000
    training_fd = {inputs: train_features, label: train_targets["cnt"][:,None]}
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in tqdm(range(hm_epochs)):      
            _, c = sess.run([optimizer, loss], feed_dict = training_fd)
            if (epoch % 1000 == 0 or epoch == 9999):
                print("Epoch", epoch, "completed out of", hm_epochs, "loss:", c)

In [16]:
train_two_hidden_nn(inputs, num_features)

  0%|          | 8/10000 [00:00<05:47, 28.79it/s]

Epoch 0 completed out of 10000 loss: 3.27467


 10%|█         | 1006/10000 [00:31<04:35, 32.60it/s]

Epoch 1000 completed out of 10000 loss: 0.365089


 20%|██        | 2005/10000 [01:02<04:11, 31.75it/s]

...

Epoch 8000 completed out of 10000 loss: 0.0929927


 90%|█████████ | 9004/10000 [04:36<00:30, 32.85it/s]

Epoch 9000 completed out of 10000 loss: 0.0826212


100%|██████████| 10000/10000 [05:07<00:00, 33.23it/s]

Epoch 9999 completed out of 10000 loss: 0.0763712


## Refactor code to allow for saving and loading models

In [17]:
num_features = train_features.shape[1]
labels_count = 1

inputs = tf.placeholder(tf.float32, [None, num_features])
label = tf.placeholder(tf.float32, [None, labels_count])

# layer 1
num_hidden_1 = 8
weights_i_h1 = tf.Variable(tf.truncated_normal((num_features, num_hidden_1)))
biases_i_h1 = tf.Variable(tf.zeros(num_hidden_1))
logits_i_h1 = tf.matmul(inputs, weights_i_h1) + biases_i_h1 # when to use tf.add?
sigmoid_i_h1 = tf.nn.sigmoid(logits_i_h1, name=None)

#layer 2
num_hidden_2 = 8
weights_h1_h2 = tf.Variable(tf.truncated_normal((num_hidden_1, num_hidden_2)))
biases_h1_h2 = tf.Variable(tf.zeros(num_hidden_2))
logits_h1_h2 = tf.matmul(sigmoid_i_h1, weights_h1_h2) + biases_h1_h2 # when to use tf.add?
sigmoid_h1_h2 = tf.nn.sigmoid(logits_h1_h2, name=None)

#output
weights_h2_o = tf.Variable(tf.truncated_normal((num_hidden_2, labels_count)))
biases_h2_o = tf.Variable(tf.zeros(labels_count))
prediction = tf.matmul(sigmoid_h1_h2, weights_h2_o) + biases_h2_o

saver = tf.train.Saver()

In [18]:
loss = tf.reduce_mean(tf.squared_difference(prediction, label))
optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)    # default learning_rate = 0.001

# cycles of feedforwards + backprop
hm_epochs = 10000
training_fd = {inputs: train_features, label: train_targets["cnt"][:,None]}
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in tqdm(range(hm_epochs)):      
        _, c = sess.run([optimizer, loss], feed_dict = training_fd)
        if (epoch % 1000 == 0 or epoch == 9999):
            print("Epoch", epoch, "completed out of", hm_epochs, "loss:", c)
    save_path = saver.save(sess, "model.ckpt")
    print(save_path)

  0%|          | 3/10000 [00:00<07:10, 23.24it/s]

Epoch 0 completed out of 10000 loss: 1.04927


 10%|█         | 1005/10000 [00:30<04:30, 33.30it/s]

Epoch 1000 completed out of 10000 loss: 0.42554


 20%|██        | 2005/10000 [01:00<04:03, 32.83it/s]

...

Epoch 8000 completed out of 10000 loss: 0.0891278


 90%|█████████ | 9005/10000 [04:33<00:30, 33.00it/s]

Epoch 9000 completed out of 10000 loss: 0.0843493


100%|██████████| 10000/10000 [05:02<00:00, 33.06it/s]


Epoch 9999 completed out of 10000 loss: 0.0809066
model.ckpt


In [19]:
# load model from checkpoint and calculate loss for test data
with tf.Session() as sess:
    testing_fd = {inputs: test_features, label: test_targets["cnt"][:,None]}
    saver.restore(sess, "./model.ckpt")
    print(sess.run(loss, feed_dict=testing_fd))


0.277086
